## 1. Setup


### 1.1 Importing Dependencies


In [32]:
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
import os
import webbrowser
import pytube
import matplotlib.pyplot as plt
import urllib.request

## 2. Creating Dataset


### 2.1 Creating directories


In [33]:
os.makedirs('data', exist_ok=True)
os.makedirs('data/images', exist_ok=True)
words_path = os.path.join('data', 'words.csv')

### 2.2 Creating Word List


In [48]:
word_list = []

df = pd.DataFrame({'word': word_list})
df.to_csv(words_path, index=False)

### 2.3 Youtube Scraping


In [34]:
def scrape_youtube_videos(query):
    api_key = os.environ.get("YOUTUBE_API_KEY")
    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

    video_links = []
    video_ids = []
    next_page_token = None

    while True:
        search_response = youtube.search().list(
            q=query,
            type='video',
            part='id',
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in search_response['items']:
            video_links.append(
                'https://www.youtube.com/watch?v=' + item['id']['videoId'])
            video_ids.append(item['id']['videoId'])

        next_page_token = search_response.get('nextPageToken')

        if not next_page_token:
            break

    df = pd.DataFrame({'link': video_links, 'id': video_ids})
    p = os.path.join('data', query + '.csv')
    df.to_csv(p, index=False)

In [35]:
wd = pd.read_csv(words_path)
wd = wd['word'].tolist()

In [41]:
for word in wd:
    scrape_youtube_videos(word)

### 2.4 Creating Unique Dataset


In [6]:
for word in wd:
    p = os.path.join('data', word + '.csv')
    if os.path.exists(p):
        df = pd.read_csv(p)
        p2 = os.path.join('data', 'links.csv')
        df.to_csv(p2, mode='a', header=False, index=False)

In [36]:
df = pd.read_csv('data/links.csv')
df.columns = ['index', 'link', 'id']
df.to_csv('data/links.csv', index=False)

In [37]:
df = pd.read_csv('data/links.csv')
df = df.drop('index', axis=1)
# df.head()

In [50]:
video_id = df['id']
video_id = video_id.sample(5)
# video_id

## 3. Visualizing Dataset


### 3.1 Playing Video in Browser


In [39]:
def play_youtube_video(id):
    url = f"https://www.youtube.com/watch?v={id}"
    webbrowser.open(url)

In [91]:
for id in video_id:
    play_youtube_video(id)

### 3.2 Viewing Thumbnails


In [47]:
def plot_thumbnail(id):
    url = f"https://www.youtube.com/watch?v={id}"
    yt = pytube.YouTube(url)
    thumbnail_url = yt.thumbnail_url
    p = os.path.join('data', 'images', 'thumbnail.jpg')
    urllib.request.urlretrieve(thumbnail_url, p)
    thumbnail_image = plt.imread(p)
    plt.imshow(thumbnail_image)
    plt.axis('off')
    plt.show()

In [49]:
# for id in video_id:
#     plot_thumbnail(id)

## 4. Reporting Videos


In [42]:
res_path = os.path.join('data', 'res.csv')

In [43]:
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [44]:
def report_video(id):
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret.json"
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(api_service_name, api_version,credentials=credentials)
    try:
        request = youtube.videos().reportAbuse(
            body={
                "videoId": id,
                "reasonId": "N", 
            }
        )
        request.execute()
        df = pd.DataFrame({'id': [id], 'response': "success"})
        df.to_csv(res_path, mode='a', header=False, index=False)
    except googleapiclient.errors.HttpError as e:
        df = pd.DataFrame({'id': [id], 'response': "failed"})
        df.to_csv(res_path, mode='a', header=False, index=False)

In [52]:
for id in video_id:
    report_video(id)